In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from utils import evaluate_gap,collate_batch_ptr
from local_search import local_search_predict
from torch.utils.data import DataLoader
from tqdm import tqdm
import torch.nn as nn
import torch

from multiprocessing import get_context


%reload_ext autoreload
%autoreload 2


In [5]:
DEVICE = "cuda" if torch.cuda.is_available() else  "cpu"

# Evaluation Metric

**Gap**: $100*\frac{model\_time - optimal\_time}{optimal\_time}$

# Load Data

In [8]:
instances = np.load('data/instances.npy')
instances_orders = np.load('data/instances_orders_unpadded.npy')

X,y = instances,instances_orders

In [7]:
y[1234]

array([ 0,  3,  2,  7,  4,  5, 10, 15, 11, 17,  1,  8, 13, 18,  9,  6, 16,
       19, 12, 14, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1])

# Unpad data

In [9]:
from utils import unpad

for seq in y:
    seq = unpad(seq)

In [38]:
np.save('data/instances_orders_unpadded.npy',y)

# Split data

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
train_dataset = list(zip(X_train,y_train))
test_dataset = list(zip(X_test,y_test))

# Solving TSP with Local Search

In [11]:
test_dataloader = DataLoader(test_dataset,batch_size=256)

In [16]:
y_preds = []
total_gaps = 0
total = 0

for x,y in tqdm(test_dataloader,total = len(test_dataloader)):
    total += len(y)

    with get_context("spawn").Pool() as pool:
                        preds = pool.map(
                            local_search_predict,x)
                        gaps = pool.starmap(
                                evaluate_gap, list(zip(preds,y,x))
                                
                        )
    y_preds.append(preds)
    total_gaps+= np.sum(gaps)
    
print(f"AVERAGE GAP FOR LOCAL SEARCH SOLVER: {total_gaps/total}")




100%|██████████| 43/43 [03:47<00:00,  5.29s/it]

AVERAGE GAP FOR LOCAL SEARCH SOLVER: 10.819183554028097


# PointerNets

In [12]:
import torch
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torch.autograd import Variable
from torch.utils.data import DataLoader

import numpy as np
import argparse
from tqdm import tqdm

from pointernet import PointerNet
from Data_Generator import TSPDataset

In [13]:
LR = 0.1
BATCH_SIZE = 16
NUM_EPOCHS = 1

In [35]:
model = PointerNet(128,
                   512,
                   2,
                   0.1,
                   False)

test_loader = DataLoader(test_dataset,
                        batch_size=10,
                        shuffle=True,
                        collate_fn=collate_batch_ptr)

criterion = torch.nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.SGD(model.parameters(),lr = LR)

In [36]:
def train_ptr(dataloader, model, optimizer, criterion, epoch):
    model.train()
    total_loss,total_gap, total_count = 0,0,0
    log_interval = 100

    for idx, (x, y) in tqdm(enumerate(dataloader)):
        optimizer.zero_grad()

        y_pred,pointers = model(x)
        y_pred =  y_pred.contiguous().view(-1, y_pred.size()[-1])

        # Get the loss
        loss = criterion(y_pred,y.view(-1))
        total_loss  += loss.item()
        # Do back propagation
        loss.backward()
        # Clip the gradients at 0.1
        nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        # Do an optimization step
        optimizer.step()
        for i,pred in enumerate(pointers.detach().numpy()):
            total_gap += evaluate_gap(pred,y[i].detach().numpy(),x[i])
        total_count += len(y)
        if idx % log_interval == 0 and idx > 0:
            print(
                "| epoch {:3d} | {:5d}/{:5d} batches "
                "| gap {:8.3f}".format(epoch, idx, len(dataloader), total_gap / total_count)
                "| loss {:8.3f}".format(epoch, idx, len(dataloader), total_loss / total_count)
            )
            total_gap, total_count = 0, 0

In [37]:
train_ptr(test_loader,model,optimizer,criterion,1)

101it [01:01,  1.60it/s]

| epoch   1 |   100/ 1085 batches | gap  250.937


201it [02:04,  1.68it/s]

| epoch   1 |   200/ 1085 batches | gap  363.790


233it [02:23,  1.62it/s]


KeyboardInterrupt: 

In [20]:
dataset = TSPDataset(10,
                     5)

dataloader = DataLoader(dataset,
                        2,
                        shuffle=True,
                        num_workers=4)


iterator = tqdm(dataloader, unit='Batch')

for i_batch, sample_batched in enumerate(iterator):
        train_batch = Variable(sample_batched['Points'])
        target_batch = Variable(sample_batched['Solution'])

        o, p = model(train_batch)
        o = o.contiguous().view(-1, o.size()[-1])
        target_batch = target_batch.view(-1)

        break

Data points 2/10:   0%|          | 0/10 [00:00<?, ?data/s]

  0%|          | 0/5 [00:00<?, ?Batch/s]/Users/aymenkallala/miniconda3/envs/instadeep/lib/python3.8/site-packages/torch/nn/modules/module.py:1511: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
/Users/aymenkallala/Desktop/Columbia/instadeep_assignment/pointernet.py:246: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/IndexingUtils.h:28.)
  decoder_input = embedded_inputs[embedding_mask.data].view(batch_size, self.embedding_dim)
  0%|          | 0/5 [00:11<?, ?Batch/s]


In [30]:
print(train_batch.shape)
print(target_batch.shape)
print(o.shape)

torch.Size([2, 5, 2])
torch.Size([10])
torch.Size([10, 5])


torch.Size([10, 5])

In [10]:
test_loader = DataLoader(test_dataset,
                        batch_size=16,
                        shuffle=True,
                        collate_fn=collate_batch_ptr)

In [33]:
for x,y,dm in test_loader:
    o, p = model(x)
    o = o.contiguous().view(-1, o.size()[-1])
    y = y.view(-1)

    print(y.shape)
    print(o.shape)
    break
    

torch.Size([800])
torch.Size([800, 50])


In [17]:
o.contiguous().view(-1, o.size()[-1]).shape

torch.Size([800, 50])

In [15]:
x

tensor([[[ 0.0535,  0.7385],
         [ 0.4729,  0.2111],
         [ 0.3706,  0.9471],
         ...,
         [-1.0000, -1.0000],
         [-1.0000, -1.0000],
         [-1.0000, -1.0000]],

        [[ 0.8966,  0.8418],
         [ 0.7322,  0.2105],
         [ 0.0556,  0.2626],
         ...,
         [ 0.2081,  0.0039],
         [-1.0000, -1.0000],
         [-1.0000, -1.0000]],

        [[ 0.4766,  0.0024],
         [ 1.0000,  0.3169],
         [ 0.1034,  0.0026],
         ...,
         [-1.0000, -1.0000],
         [-1.0000, -1.0000],
         [-1.0000, -1.0000]],

        ...,

        [[ 1.0000,  0.7893],
         [ 0.4227,  0.4748],
         [ 0.1039,  0.1589],
         ...,
         [-1.0000, -1.0000],
         [-1.0000, -1.0000],
         [-1.0000, -1.0000]],

        [[ 0.9445,  0.3713],
         [ 0.5283,  0.9464],
         [ 0.3177,  0.4223],
         ...,
         [-1.0000, -1.0000],
         [-1.0000, -1.0000],
         [-1.0000, -1.0000]],

        [[ 0.1066,  0.4695],
       

# Transformer Net

In [86]:
from transformernet import TSP_net

In [89]:
test_loader = DataLoader(test_dataset,
                        batch_size=16,
                        shuffle=True,
                        collate_fn=collate_batch_ptr)

In [90]:
model_train = TSP_net(2, 128, 512, 
              6, 2, 8, 1000,
              False)

In [92]:
for x, y in test_loader:
    print(x.shape)
    y_pred = model_train(x)
    print(y_pred[0][1])
    print(y[1])
    print(x[1])
    break

torch.Size([16, 50, 2])
tensor([37, 30,  3, 28,  5, 26, 27,  1, 24, 29, 19, 21, 31,  9, 20, 22, 35, 23,
         4, 14, 11, 34,  7, 16, 33,  2, 17,  6, 12,  8, 32, 36, 10, 15, 18, 45,
        40, 25, 13,  0, 48, 43, 42, 46, 39, 47, 38, 44, 41, 49])
tensor([ 0, 10, 15, 31, 27,  3, 13, 18, 22, 29, 24, 12, 17, 14,  2, 33, 23, 36,
        20,  4, 16,  1, 11, 26, 30, 28,  9, 32, 21,  6, 34, 35,  7,  8, 25, 19,
         5, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1])
tensor([[ 0.0043,  0.0529],
        [ 0.5244,  0.9983],
        [ 0.2644,  0.6819],
        [ 0.1576,  0.4723],
        [ 0.3675,  0.9451],
        [ 0.1551,  0.1053],
        [ 0.7382,  0.5765],
        [ 0.7900,  0.0018],
        [ 0.7367,  0.0015],
        [ 0.9485,  0.9472],
        [ 0.0513,  0.3122],
        [ 0.5290,  0.7354],
        [ 0.5295,  0.3148],
        [ 0.2128,  0.4243],
        [ 0.3147,  0.4225],
        [ 0.1025,  0.3678],
        [ 0.4738,  1.0000],
        [ 0.5299,  0.4747],
        [ 0.2073,  0.2